# pipelines.agent.model

> Agent-friendly wrapper around and LLM instance

In [ ]:
# | default_exp pipelines.agent.model

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

from typing import Any, Dict, List, Optional
import json
import re
from smolagents.models import Model, ChatMessage, MessageRole
from smolagents.models import get_tool_call_from_text, remove_stop_sequences
from smolagents import get_clean_message_list, tool_role_conversions
from enum import Enum
import onprem

In [ ]:
# | export


class AgentModel(Model):
    """
    A smolagents Model implementation that wraps an onprem LLM instance.
    
    This adapter allows onprem LLM instances to be used with smolagents Agents.
    
    Parameters:
        llm (LLM): An instance of onprem.llm.base.LLM
        model_id (str, optional): An identifier for the model
        **kwargs: Additional keyword arguments to pass to the parent Model class
    """
    
    def __init__(
        self,
        llm: onprem.LLM,
        model_id: Optional[str] = None,
        **kwargs
    ):
        # Initialize the parent Model class
        super().__init__(
            model_id=model_id or f"onprem-{llm.model_name}",
            **kwargs
        )
        # Store the LLM instance
        self.llm = llm
        
    def generate(
        self,
        messages: List[Dict[str, Any] | ChatMessage],
        stop_sequences: Optional[List[str]] = None,
        response_format: Optional[Dict[str, str]] = None,
        tools_to_call_from: Optional[List[Any]] = None,
        **kwargs
    ) -> ChatMessage:
        """
        Process the input messages and return the model's response.
        
        Parameters:
            messages: A list of message dictionaries to be processed.
            stop_sequences: A list of strings that will stop generation if encountered.
            response_format: The response format to use in the model's response.
            tools_to_call_from: A list of tools that the model can use.
            **kwargs: Additional keyword arguments to pass to the LLM.
            
        Returns:
            ChatMessage: A chat message object containing the model's response.
        """
        # Convert smolagents messages to a format that onprem LLM can use
        messages = self.clean(messages)

        #messages = self._process_messages(messages)
        
        # Call the LLM with the processed messages
        response = self.llm.prompt(
            messages,
            stop=stop_sequences or [],
            **kwargs
        )
        #print('START R')
        #print(response)
        #print('END R')

        # Remove stop sequences from LLM output
        if stop_sequences is not None:
            response = remove_stop_sequences(response, stop_sequences)
        
        # Create and return a ChatMessage with the response
        message =  ChatMessage(
            role=MessageRole.ASSISTANT,
            content=response,
            raw={"response": response},
            token_usage=None  # onprem LLM doesn't track tokens in a way we can use here
        )
        if tools_to_call_from:
            try:
                message.tool_calls = [
                    get_tool_call_from_text(
                        re.sub(r".*?Action:(.*?\n\}).*", r"\1", response, flags=re.DOTALL), self.tool_name_key, self.tool_arguments_key
                    )
                ]
            except:
                message.content = self._build_final_answer_json(response)
        return message


    def clean(self, messages):
        """
        Gets a clean message list.

        Args:
            messages: input messages

        Returns:
            clean messages
        """

        # Get clean message list
        messages = get_clean_message_list(messages, role_conversions=tool_role_conversions, flatten_messages_as_text=self.flatten_messages_as_text)

        # Ensure all roles are strings and not enums for compability across LLM frameworks
        for message in messages:
            if "role" in message:
                message["role"] = message["role"].value if isinstance(message["role"], Enum) else message["role"]

        return messages


    def _process_messages(self, messages: List[Dict[str, Any] | ChatMessage]) -> str:
        """
        Convert smolagents messages to a format suitable for onprem LLM.
        
        For now, this concatenates all messages into a single string prompt.
        
        Parameters:
            messages: A list of message dictionaries or ChatMessage objects.
            
        Returns:
            str: A formatted prompt string for the LLM.
        """
        # Process each message and combine them
        processed_parts = []
        
        for msg in messages:
            # Handle ChatMessage objects
            if isinstance(msg, ChatMessage):
                role = msg.role
                content = msg.content or ""
                
                # Handle tool calls if present
                if msg.tool_calls:
                    tool_calls_str = json.dumps([tc.dict() for tc in msg.tool_calls], indent=2)
                    content = f"{content}\nTool Calls: {tool_calls_str}"
            else:
                # Handle dictionary format
                role = msg["role"]
                content = msg.get("content", "")
                
                # Handle tool calls if present in dictionary format
                if "tool_calls" in msg and msg["tool_calls"]:
                    tool_calls_str = json.dumps(msg["tool_calls"], indent=2)
                    content = f"{content}\nTool Calls: {tool_calls_str}"
            
            # Format based on role
            if role == MessageRole.USER:
                processed_parts.append(f"User: {content}")
            elif role == MessageRole.ASSISTANT:
                processed_parts.append(f"Assistant: {content}")
            elif role == MessageRole.SYSTEM:
                processed_parts.append(f"System: {content}")
            elif role == MessageRole.TOOL_CALL:
                processed_parts.append(f"Tool Call: {content}")
            elif role == MessageRole.TOOL_RESPONSE:
                processed_parts.append(f"Tool Response: {content}")
        
        # Combine all parts with newlines
        return "\n\n".join(processed_parts)


    def _build_final_answer_json(self, answer: str) -> str:
        data = {
            "name": "final_answer",
            "arguments": {
            "answer": answer
            }
        }
        import json
        return json.dumps(data, ensure_ascii=False, indent=2)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()